In [ ]:
import mlrun

# Create the 'LLM chatbot' AIPC project

In [ ]:
project = mlrun.get_or_create_project(
    name="llm-dialog-aipc",
    context="./",
    parameters={
        "source": "https://github.com/smartcommunitylab/ai_product_card_templates/tree/main/aipc_examples/llm",
        "default_image": "mlrun/mlrun",
        "requirements_file": "./requirements.txt",
        "metadata_path": "../../metadata/",
    },
)

# Training functions

In [ ]:
# Training function
project.set_function(
    name="training",
    func='functions/training.py',
    handler='train',
    kind="job",
)

# Serving function

In [ ]:
#Serving function
serving_fn = mlrun.code_to_function("serving-5760", filename="serving/serve.py", kind="serving", image="mlrun/mlrun")
serving_fn.spec.build.commands = [
    "pip install -r requirements.txt",
    "pip uninstall apex"
]
serving_fn.add_model(
    "pythia-5760",
    model_path=" ",
    model_name="EleutherAI/pythia-1.4b-deduped",
    adapter_path='../models/checkpoints/pythia-1.4b-finetuned-orig/checkpoint-9360',
    class_name="ChatBot"
)
project.deploy_function(serving_fn)
project.save()

In [ ]:
# test function
server = serving_fn.to_mock_server()

In [ ]:
def chat_with_model(skip_special_tokens=True, max_length=64):
    conversation_history = []

    while True:
        # Get user input
        user_input = input("You: ") + f" <|endofturn|>"
        # Add user input to conversation history
        conversation_history.append(user_input)

        # Generate response
        sample = {
            "row": ' '.join(conversation_history), 
            "skip_special_tokens": skip_special_tokens, 
            "max_length": max_length
        }
        response= serving_fn.invoke(path=f"/v2/models/pythia-5760/infer", body=sample)
        response = response.split("<|endofturn|>")[0] 
        response_hist = response.split("<|endofturn|>")[0] + f" <|endofturn|>"
        # Print and add response to conversation history
        # print("history:", conversation_history)
        print("Model:", response)
        conversation_history.append(response_hist)

        # Check for exit condition
        if user_input.lower() == "exit":
            print("\n\n\n\n#### ALL DIALOGUE HISTORY ####\n\n", '\n'.join(conversation_history))

In [ ]:
chat_with_model(skip_special_tokens=False, max_length=128)